In [1]:
import pandas as pd

In [2]:
vacc_df = pd.read_csv("../data/raw/cowin_vaccine_data_statewise.csv")
pop_df = pd.read_csv("../data/raw/state_wise_population.csv")
cases_df = pd.read_csv("../data/raw/state_wise_daily.csv")

In [3]:
pop_df.columns = pop_df.columns.str.strip()

In [4]:
for col in ["Total 2020", "Male 2020", "Female 2020", "Total 2021", "Male 2021", "Female 2021"]:
    pop_df[col] = (
        pop_df[col]
        .astype(str)
        .str.replace(",", "", regex=True)
        .astype("Int64")
    )

pop_df = pop_df[["State", "Total 2021"]]

alias = {"UK": "UT", "OD": "OR", "DD": "DN"}


In [5]:
pop_df = pop_df[["State", "Total 2021"]]

In [6]:
state_mapping = {
    "Andaman and Nicobar Islands": "AN", "Andhra Pradesh": "AP", "Arunachal Pradesh": "AR",
    "Assam": "AS", "Bihar": "BR", "Chandigarh": "CH", "Chhattisgarh": "CG",
    "Dadra and Nagar Haveli and Daman and Diu": "DD", "Delhi": "DL", "Goa": "GA",
    "Gujarat": "GJ", "Haryana": "HR", "Himachal Pradesh": "HP", "Jammu and Kashmir": "JK",
    "Jharkhand": "JH", "Karnataka": "KA", "Kerala": "KL", "Ladakh": "LA", "Lakshadweep": "LD",
    "Madhya Pradesh": "MP", "Maharashtra": "MH", "Manipur": "MN", "Meghalaya": "ML",
    "Mizoram": "MZ", "Nagaland": "NL", "Odisha": "OD", "Puducherry": "PY",
    "Punjab": "PB", "Rajasthan": "RJ", "Sikkim": "SK", "Tamil Nadu": "TN",
    "Telangana": "TG", "Tripura": "TR", "Uttar Pradesh": "UP", "Uttarakhand": "UK",
    "West Bengal": "WB"
}

In [7]:
pop_df["State"] = pop_df["State"].map(lambda x: state_mapping.get(x, x))
vacc_df["State"] = vacc_df["State"].map(lambda x: state_mapping.get(x, x))

pop_df["State"] = pop_df["State"].replace(alias)
vacc_df["State"] = vacc_df["State"].replace(alias)


In [8]:
vacc_df = vacc_df[["Updated On", "State", "Total Doses Administered"]]

non_state = {"India", "State Unassigned", "Other", "Unassigned"}
vacc_df = vacc_df[~vacc_df["State"].isin(non_state)]

vacc_df["Updated On"] = pd.to_datetime(vacc_df["Updated On"], dayfirst=True, errors="coerce")
vacc_latest = (
    vacc_df.sort_values(["State", "Updated On"])\
            .drop_duplicates(subset=["State"], keep="last")
)

vacc_latest = vacc_latest[["State", "Total Doses Administered"]]


In [9]:
vacc_merged = pd.merge(vacc_latest, pop_df, on="State", how="left")
vacc_merged = vacc_merged.dropna(subset=["Total 2021"])
vacc_merged["vaccination_pct"] = (
    vacc_merged["Total Doses Administered"] / vacc_merged["Total 2021"] * 100
)


In [10]:
cases_df.iloc[:, 3:] = cases_df.iloc[:, 3:].apply(pd.to_numeric, errors="coerce")

In [11]:
value_cols = cases_df.columns[3:]
cases_pivot = (
    cases_df.pivot_table(
        index="Date_YMD",
        columns="Status",
        values=value_cols,
        aggfunc="sum",
    )
    .reset_index()
)


In [12]:
cases_pivot.columns = [
    "_".join(col).strip("_") if isinstance(col, tuple) else col
    for col in cases_pivot.columns.values
]

In [13]:
vacc_merged.to_csv("../data/processed/cleaned_vaccination_data.csv", index=False)
cases_pivot.to_csv("../data/processed/cleaned_case_data.csv", index=False)

In [14]:
print("\n=== Vaccination Data ===")
print(vacc_merged.head())

print("\n=== Case Data ===")
print(cases_pivot.head())


=== Vaccination Data ===
  State  Total Doses Administered  Total 2021  vaccination_pct
0    AN                    991264      401000       247.198005
1    AP                 110957429    52849000       209.951804
2    AR                   1924584     1537000       125.216916
3    AS                  50335778    35155000       143.182415
4    BR                 157292508   123695000       127.161573

=== Case Data ===
     Date_YMD  AN_Confirmed  AN_Deceased  AN_Recovered  AP_Confirmed  \
0  01/01/2021             1            0             4           326   
1  01/01/2022             5            0             2           176   
2  01/01/2023             0            0             0             0   
3  01/02/2021             0            0             4            64   
4  01/02/2022            32            0            62          6213   

   AP_Deceased  AP_Recovered  AR_Confirmed  AR_Deceased  AR_Recovered  ...  \
0            0           350             0            0           

In [15]:
print(vacc_df.isna().sum())
print(cases_df.isna().sum().sum()) 

Updated On                  0
State                       0
Total Doses Administered    0
dtype: int64
0


In [16]:
vacc_df.to_csv("../data/processed/cleaned_vaccinations.csv", index=False)
cases_df.to_csv("../data/processed/cleaned_cases.csv", index=False)